In [14]:
import gensim
import string
import nltk
import sparknlp

from pyspark.ml.feature import HashingTF, IDF, Tokenizer
from pyspark.ml import Pipeline
from pyspark.sql import functions as F
from gensim.parsing.preprocessing import STOPWORDS
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *
from nltk.tokenize import word_tokenize

nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package wordnet to
[nltk_data]     /common/home/ac1771/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     /common/home/ac1771/nltk_data...


True

In [2]:
df_truncated = spark.read.json('/common/users/shared/cs543_fall22_group3/realnews/realnews_truncated')

In [15]:
def lemmatize_stemming(text):
    return WordNetLemmatizer().lemmatize(text, pos='v')

# Remove punctuation, stop words, and lower case the letters
def remove_stopwords(line):
    tokens = word_tokenize(line)
    remove_stopwords = [lemmatize_stemming(t.lower()) for t in tokens if not t in STOPWORDS and not t in string.punctuation and len(t) > 2]
    return remove_stopwords

remove_stopwords_udf = F.udf(lambda z: remove_stopwords(z))
stemmed_df = df_truncated.withColumn("cleaned_summary", remove_stopwords_udf(F.col("coalesced_col")))

In [ ]:
stemmed_df.write.mode("Overwrite").json('/common/users/shared/cs543_fall22_group3/realnews/realnews_processed')